In [6]:
{-# LANGUAGE OverloadedStrings, TemplateHaskell, DisambiguateRecordFields #-}
import Data.Maybe
import Control.Monad
import Text.Feed.Import
import Text.Feed.Types
import Text.Atom.Feed as F(Feed, Entry(..), EntryContent, Category, Generator, Link, Source, Person, InReplyTo, InReplyTotal, TextContent)
import Control.Lens hiding (makeClassy)
import qualified Language.Haskell.TH.Syntax as TH
import qualified Language.Haskell.TH.Lib as TH
import qualified Text.XML.Light as XML
import Text.XML.Light(parseXMLDoc)

makeClassy = makeLensesWith $ classyRules & lensField .~ (\_ _ name -> [TopName $ TH.mkName $ TH.nameBase name])
makeClassy' = makeLensesWith $ classyRules & lensField .~ (\_ _ name -> [TopName $ TH.mkName $ "_" ++ TH.nameBase name ])


makePrisms ''XML.Content
makeClassy ''XML.Element
makeClassy ''XML.Attr
makeClassy ''XML.CData
makeClassy' ''XML.QName

makeClassy ''F.Feed
makeClassy ''F.Entry
makeClassy ''F.Category
makeClassy ''F.Generator
makeClassy ''F.Link
makeClassy ''F.Source
makeClassy ''F.Person
makeClassy ''F.InReplyTo
makeClassy ''F.InReplyTotal
makePrisms ''F.EntryContent
makePrisms ''F.TextContent

In [7]:
{-# LANGUAGE RecordWildCards #-}
declareLensesWith (classyRules & generateLazyPatterns .~ True) [d|
  data JiraActivity = JiraActivity {_person::Person,
                                    _activityUID::String,
                                    _summary::String}
    deriving Show
  |]

In [9]:
AtomFeed feed <- parseFeedFromFile "file.rss"


In [ ]:
entryToJiraActivity :: Entry -> JiraActivity
entryToJiraActivity (Entry {..}) = JiraActivity {_person=head entryAuthors
                                                 _summary=entryTitle
                                                 _content=}
                                      cdData

In [4]:
text = flip map (feed ^. feedEntries) (\x ->
            x ^.. entryContent
              ._Just
              . _HTMLContent 
              . to parseXMLDoc 
              ._Just    
              . elContent
              . traverse
              . deepOf (_Elem.elContent.traverse) (_Text.cdData)) 

forM_ text $ \entryContent -> do
    putStrLn $ concat entryContent
    putStrLn "------------------------------------------------------------"
            


  Added the Affects Version '4.2.NEXT'
  Added the Affects Version '4.0'
  Added the Affects Version '4.1'
  Added the Fix Version '4.2.NEXT'

------------------------------------------------------------

   4.2: merged,

34e56609d88650a46dff54c3919949a615eaeb07 (10 hours ago) laszlo.attila.toth@balabit.com widgets: button: properly warning spans, needed for multiple warnings
7e243b0131c1f1b82d578115312a884e7625e609 (10 hours ago) laszlo.attila.toth@balabit.com basic system control: Starting/stopping all traffic ignores disabled protos
e8ac0f76f0fea7aac1b29867b1fe8974d21f65b1 (10 hours ago) laszlo.attila.toth@balabit.com templates: basic-system-control: add warning to disabled services
ba9ecac00fe6d0cae93279e3f9ec9e34d90233dc (10 hours ago) laszlo.attila.toth@balabit.com widgets: button: may have warning icon
dcca3105eb7151eb1d2466f79c34c60fd2d1028e (10 hours ago) laszlo.attila.toth@balabit.com js_scb/basic-system.js: warning: refer Global Options page
7f3c2194eeaf9bbc42df2fb7a8846218

In [1]:
:info _feedEntries

Not in scope: ‘_feedEntries’

In [2]:
:info elContent

Not in scope: ‘elContent’

In [3]:
:t feed ^.. feedEntries 
      . (traverse 
      . entryContent
      ._Just
      . _HTMLContent 
      . to parseXMLDoc 
      ._Just
      . elContent
      .traverse
      ._Elem)


Not in scope: ‘feed’


Not in scope: ‘feedEntries’


Not in scope: ‘^..’


Not in scope: ‘entryContent’


Not in scope: ‘to’


Not in scope: ‘parseXMLDoc’


Not in scope: ‘elContent’

In [4]:
infixl 1 &?
(&?) :: a -> (a -> Maybe a) -> a
value &? setter = case setter value of
                        Nothing -> value
                        Just new_value -> new_value


In [ ]:
id id id id 4

In [ ]:
-- [Just [Just [Nothing, Just 4]], Nothing, Just [Just [Just 5]]] ^.. deepOf (traverse._Just.traverse) _Just


In [ ]:
:info (.)